# Importing Necessary Libraries

In [ ]:
import re
import string

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
plt.style.use('fivethirtyeight')

import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer

import json
import warnings
warnings.filterwarnings('ignore')
import base64
import io

import codecs
from IPython.display import HTML

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

sns.set(font_scale=1.3)
nltk.download('omw-1.4')

!pip install python-docx


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 7.2 MB/s eta 0:00:00


In [ ]:
# Exporting dataframes to docs

'''
df = dataframe_name.head(1)

import docx

# Initialise the Word document
doc = docx.Document()
# Initialise the table
t = doc.add_table(rows=1, cols=df.shape[1])
# Add borders
t.style = 'TableGrid'
# Add the column headings
for j in range(df.shape[1]):
    t.cell(0, j).text = df.columns[j]
# Add the body of the data frame
for i in range(df.shape[0]):
    row = t.add_row()
    for j in range(df.shape[1]):
        cell = df.iat[i, j]
        row.cells[j].text = str(cell)
# Save the Word doc
doc.save('table_1.docx')
'''

"\ndf = dataframe_name.head(1)\n\nimport docx\n\n# Initialise the Word document\ndoc = docx.Document()\n# Initialise the table\nt = doc.add_table(rows=1, cols=df.shape[1])\n# Add borders\nt.style = 'TableGrid'\n# Add the column headings\nfor j in range(df.shape[1]):\n    t.cell(0, j).text = df.columns[j]\n# Add the body of the data frame\nfor i in range(df.shape[0]):\n    row = t.add_row()\n    for j in range(df.shape[1]):\n        cell = df.iat[i, j]\n        row.cells[j].text = str(cell)\n# Save the Word doc\ndoc.save('table_1.docx')\n"

# Read Dataset

In [ ]:
credits = pd.read_csv('/content/drive/MyDrive/DBMS_project/The_Movie_Dataset/credits.csv')
keywords = pd.read_csv('/content/drive/MyDrive/DBMS_project/The_Movie_Dataset/keywords.csv')
movies = pd.read_csv('/content/drive/MyDrive/DBMS_project/The_Movie_Dataset/movies_metadata.csv').\
                     drop(['belongs_to_collection', 'homepage', 'imdb_id', 'poster_path', 'status', 'title', 'video'], axis=1).\
                     drop([19730, 29503, 35587]) # Incorrect data type

movies['id'] = movies['id'].astype('int64')

df = movies.merge(keywords, on='id').\
    merge(credits, on='id')

df['original_language'] = df['original_language'].fillna('')
df['runtime'] = df['runtime'].fillna(0)
df['tagline'] = df['tagline'].fillna('')

df.dropna(inplace=True)

df.head(2)

,adult,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,keywords,cast,crew
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",,7.7,5415.0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,6.9,2413.0,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


In [ ]:
from ast import literal_eval

def get_text(text, obj='name'):
    text = literal_eval(text)

    if len(text) == 1:
        for i in text:
            return i[obj]
    else:
        s = []
        for i in text:
            s.append(i[obj])
        return ', '.join(s)

df['genres'] = df['genres'].apply(get_text)
df['production_companies'] = df['production_companies'].apply(get_text)
df['production_countries'] = df['production_countries'].apply(get_text)
df['crew'] = df['crew'].apply(get_text)
df['spoken_languages'] = df['spoken_languages'].apply(get_text)
df['keywords'] = df['keywords'].apply(get_text)

# New columns
df['characters'] = df['cast'].apply(get_text, obj='character')
df['actors'] = df['cast'].apply(get_text)

df.drop('cast', axis=1, inplace=True)
df = df[~df['original_title'].duplicated()]
df = df.reset_index(drop=True)

In [ ]:
df.head(2)

,adult,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,...,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,keywords,crew,characters,actors
0,False,30000000,"Animation, Comedy, Family",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,...,373554033.0,81.0,English,,7.7,5415.0,"jealousy, toy, boy, friendship, friends, rival...","John Lasseter, Joss Whedon, Andrew Stanton, Jo...","Woody (voice), Buzz Lightyear (voice), Mr. Pot...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,False,65000000,"Adventure, Fantasy, Family",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,...,262797249.0,104.0,"English, Français",Roll the dice and unleash the excitement!,6.9,2413.0,"board game, disappearance, based on children's...","Larry J. Franco, Jonathan Hensleigh, James Hor...","Alan Parrish, Samuel Alan Parrish / Van Pelt, ...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."


In [ ]:
df.drop(['adult', 'budget', 'original_language','popularity', 'production_countries','release_date', 'revenue', 'runtime',
        'spoken_languages', 'tagline', 'vote_average', 'vote_count', 'characters'], axis=1, inplace=True)


In [ ]:
df.head(1)

,genres,id,original_title,production_companies,keywords,actors
0,"Animation, Comedy, Family",862,Toy Story,Pixar Animation Studios,"jealousy, toy, boy, friendship, friends, rival...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."


In [ ]:
# Exporting dataframes to docs


df = df.head(1)

import docx

# Initialise the Word document
doc = docx.Document()
# Initialise the table
t = doc.add_table(rows=1, cols=df.shape[1])
# Add borders
t.style = 'TableGrid'
# Add the column headings
for j in range(df.shape[1]):
    t.cell(0, j).text = df.columns[j]
# Add the body of the data frame
for i in range(df.shape[0]):
    row = t.add_row()
    for j in range(df.shape[1]):
        cell = df.iat[i, j]
        row.cells[j].text = str(cell)
# Save the Word doc
doc.save('table_1.docx')


In [ ]:
df.iloc[0]

genres                                          Animation, Comedy, Family
id                                                                    862
original_title                                                  Toy Story
overview                Led by Woody, Andy's toys live happily in his ...
production_companies                              Pixar Animation Studios
keywords                jealousy, toy, boy, friendship, friends, rival...
crew                    John Lasseter, Joss Whedon, Andrew Stanton, Jo...
actors                  Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...
Name: 0, dtype: object

In [ ]:
df.shape

(42373, 8)

In [ ]:
df.dtypes

genres                  object
id                       int64
original_title          object
overview                object
production_companies    object
keywords                object
crew                    object
actors                  object
dtype: object

### Coverting df to pickle for faster access

In [ ]:
df.to_pickle('pickle_file')

## Run From Here for Merged Dataframe


In [ ]:
df = pd.read_pickle(r'/content/drive/MyDrive/DBMS_project/The_Movie_Dataset/pickle_file')

In [ ]:
df.head(2)

,genres,id,original_title,overview,production_companies,keywords,crew,actors
0,"Animation, Comedy, Family",862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Pixar Animation Studios,"jealousy, toy, boy, friendship, friends, rival...","John Lasseter, Joss Whedon, Andrew Stanton, Jo...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,"Adventure, Fantasy, Family",8844,Jumanji,When siblings Judy and Peter discover an encha...,"TriStar Pictures, Teitler Film, Interscope Com...","board game, disappearance, based on children's...","Larry J. Franco, Jonathan Hensleigh, James Hor...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."


# Data Cleaning

### Check if the data contains any null values

In [ ]:
#print the number of null values in each column
df.isnull().sum()

genres                  0
id                      0
original_title          0
production_companies    0
keywords                0
actors                  0
dtype: int64

# Preprocessing movie overviews


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df.head(1)

,genres,id,original_title,production_companies,keywords,actors
0,"Animation, Comedy, Family",862,Toy Story,Pixar Animation Studios,"jealousy, toy, boy, friendship, friends, rival...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."


In [ ]:
sent_0 = df['overview'].values[0]
sent_0

KeyError: 'overview'

In [ ]:
# remove urls from text python: https://stackoverflow.com/a/40823105/4084039
sent_0 = re.sub(r"http\S+", "", sent_0)
sent_0

NameError: name 'sent_0' is not defined

In [ ]:
# https://stackoverflow.com/questions/16206380/python-beautifulsoup-how-to-remove-all-tags-from-an-element
from bs4 import BeautifulSoup

soup = BeautifulSoup(sent_0, 'lxml')
text = soup.get_text()
print(text)

Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.


In [ ]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
sent_0 = decontracted(sent_0)
sent_0

'Led by Woody, Andy is toys live happily in his room until Andy is birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy is heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.'

In [ ]:
#remove words with numbers python: https://stackoverflow.com/a/18082370/4084039
sent_0 = re.sub("\S*\d\S*", "", sent_0).strip()
print(sent_0)

Led by Woody, Andy is toys live happily in his room until Andy is birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy is heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.


In [ ]:
#remove spacial character: https://stackoverflow.com/a/5843547/4084039
sent_0 = re.sub('[^A-Za-z0-9]+', ' ', sent_0)
print(sent_0)

Led by Woody Andy is toys live happily in his room until Andy is birthday brings Buzz Lightyear onto the scene Afraid of losing his place in Andy is heart Woody plots against Buzz But when circumstances separate Buzz and Woody from their owner the duo eventually learns to put aside their differences 


In [ ]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
df.head(1)

,genres,id,original_title,overview,production_companies,keywords,crew,actors
0,"Animation, Comedy, Family",862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Pixar Animation Studios,"jealousy, toy, boy, friendship, friends, rival...","John Lasseter, Joss Whedon, Andrew Stanton, Jo...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."


In [ ]:
from bs4 import BeautifulSoup

In [ ]:
def preprocess(sentance):
     sentance = re.sub(r"http\S+", "", str(sentance))
     sentance = BeautifulSoup(sentance, 'lxml').get_text()
     sentance = decontracted(sentance)
     sentance = re.sub("\S*\d\S*", "", sentance).strip()
     sentance = re.sub('[^A-Za-z]+', ' ', sentance)
     # https://gist.github.com/sebleier/554280
     sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)

     return sentance

In [ ]:
df['overview']= df['overview'].apply(preprocess)
df.head()

,genres,id,original_title,overview,production_companies,keywords,crew,actors
0,"Animation, Comedy, Family",862,Toy Story,led woody andy toys live happily room andy bir...,Pixar Animation Studios,"jealousy, toy, boy, friendship, friends, rival...","John Lasseter, Joss Whedon, Andrew Stanton, Jo...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,"Adventure, Fantasy, Family",8844,Jumanji,siblings judy peter discover enchanted board g...,"TriStar Pictures, Teitler Film, Interscope Com...","board game, disappearance, based on children's...","Larry J. Franco, Jonathan Hensleigh, James Hor...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,"Romance, Comedy",15602,Grumpier Old Men,family wedding reignites ancient feud next doo...,"Warner Bros., Lancaster Gate","fishing, best friend, duringcreditsstinger, ol...","Howard Deutch, Mark Steven Johnson, Mark Steve...","Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,"Comedy, Drama, Romance",31357,Waiting to Exhale,cheated mistreated stepped women holding breat...,Twentieth Century Fox Film Corporation,"based on novel, interracial relationship, sing...","Forest Whitaker, Ronald Bass, Ronald Bass, Ezr...","Whitney Houston, Angela Bassett, Loretta Devin..."
4,Comedy,11862,Father of the Bride Part II,george banks recovered daughter wedding receiv...,"Sandollar Productions, Touchstone Pictures","baby, midlife crisis, confidence, aging, daugh...","Alan Silvestri, Elliot Davis, Nancy Meyers, Na...","Steve Martin, Diane Keaton, Martin Short, Kimb..."


In [ ]:
df['overview'].values[0]

'led woody andy toys live happily room andy birthday brings buzz lightyear onto scene afraid losing place andy heart woody plots buzz circumstances separate buzz woody owner duo eventually learns put aside differences'

## Preprocessing Genres

In [ ]:
df['genres']= df['genres'].apply(preprocess)
df.head()

,genres,id,original_title,production_companies,keywords,actors
0,animation comedy family,862,Toy Story,Pixar Animation Studios,"jealousy, toy, boy, friendship, friends, rival...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."


In [ ]:
df['genres'].values[0]

'animation comedy family'

## Preprocessing Keywords

In [ ]:
df['keywords']= df['keywords'].apply(preprocess)
df.head()

,genres,id,original_title,production_companies,keywords,actors
0,animation comedy family,862,Toy Story,Pixar Animation Studios,jealousy toy boy friendship friends rivalry bo...,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."


In [ ]:
df['keywords'].values[0]

'jealousy toy boy friendship friends rivalry boy next door new toy toy comes life'

## Preprocessing Production Companies

In [ ]:
df['production_companies']= df['production_companies'].apply(preprocess)
df.head()

,genres,id,original_title,production_companies,keywords,actors
0,animation comedy family,862,Toy Story,pixar animation studios,jealousy toy boy friendship friends rivalry bo...,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."


In [ ]:
df['production_companies'].values[0]

'pixar animation studios'

## Preprocessing Crew

In [ ]:
'''
df['crew']= df['crew'].apply(preprocess)
df.head()
'''

,genres,id,overview,production_companies,keywords,crew,actors
0,animation comedy family,862,led woody andy toys live happily room andy bir...,pixar animation studios,jealousy toy boy friendship friends rivalry bo...,john lasseter joss whedon andrew stanton joel ...,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,adventure fantasy family,8844,siblings judy peter discover enchanted board g...,tristar pictures teitler film interscope commu...,board game disappearance based children book n...,larry j franco jonathan hensleigh james horner...,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,romance comedy,15602,family wedding reignites ancient feud next doo...,warner bros lancaster gate,fishing best friend duringcreditsstinger old men,howard deutch mark steven johnson mark steven ...,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,comedy drama romance,31357,cheated mistreated stepped women holding breat...,twentieth century fox film corporation,based novel interracial relationship single mo...,forest whitaker ronald bass ronald bass ezra s...,"Whitney Houston, Angela Bassett, Loretta Devin..."
4,comedy,11862,george banks recovered daughter wedding receiv...,sandollar productions touchstone pictures,baby midlife crisis confidence aging daughter ...,alan silvestri elliot davis nancy meyers nancy...,"Steve Martin, Diane Keaton, Martin Short, Kimb..."


In [ ]:
'''
df['crew'].values[0]
'''

'john lasseter joss whedon andrew stanton joel cohen alec sokolow bonnie arnold ed catmull ralph guggenheim steve jobs lee unkrich ralph eggleston robert gordon mary helen leasman kim blanchette marilyn mccoppen randy newman dale e grahn robin cooper john lasseter pete docter joe ranft patsy bouge norm decarlo ash brannon randy newman roman figun davis james flamberg mary beth smith rick mackay susan bradley william reeves randy newman andrew stanton pete docter gary rydstrom karen robert jackson chris montan rich quade michael berenstein colin brady davey crockett feiten angie glocka rex grignon tom k gurney jimmy hayward hal hickel karen kiser anthony b lamolinara guionne leroy bud luckey les major glenn mcqueen mark oftedal jeff pidgeon jeff pratt steve rabatich roger rose steve segal doug sheppeck alan sperling doug sweetland david tart ken willard thomas porter mark thomas henne oren jacob darwyn peachey mitch prater brian rosen sharon calahan galyn susman william cone shelley dan

## Preprocessing actors

In [ ]:
df['actors']= df['actors'].apply(preprocess)
df.head()

,genres,id,original_title,production_companies,keywords,actors
0,animation comedy family,862,Toy Story,pixar animation studios,jealousy toy boy friendship friends rivalry bo...,tom hanks tim allen rickles jim varney wallace...


In [ ]:
df['actors'].values[0]

'tom hanks tim allen rickles jim varney wallace shawn john ratzenberger annie potts john morris erik von detten laurie metcalf r lee ermey sarah freeman penn jillette'

In [ ]:
# Exporting dataframes to docs


df = df.head(1)

import docx

# Initialise the Word document
doc = docx.Document()
# Initialise the table
t = doc.add_table(rows=1, cols=df.shape[1])
# Add borders
t.style = 'TableGrid'
# Add the column headings
for j in range(df.shape[1]):
    t.cell(0, j).text = df.columns[j]
# Add the body of the data frame
for i in range(df.shape[0]):
    row = t.add_row()
    for j in range(df.shape[1]):
        cell = df.iat[i, j]
        row.cells[j].text = str(cell)
# Save the Word doc
doc.save('table_2.docx')


## Create a pickle file of the preprocessed dataframe

In [ ]:
df.to_pickle('preprocessed_pickle_file')

## Run From Here for Preprocessed Dataframe

In [ ]:
# Run

df = pd.read_pickle(r'/content/drive/MyDrive/DBMS_project/The_Movie_Dataset/preprocessed_pickle_file')

In [ ]:
# Run

df.head(1)

,genres,id,original_title,overview,production_companies,keywords,crew,actors
0,animation comedy family,862,Toy Story,led woody andy toys live happily room andy bir...,pixar animation studios,jealousy toy boy friendship friends rivalry bo...,"John Lasseter, Joss Whedon, Andrew Stanton, Jo...",tom hanks tim allen rickles jim varney wallace...


In [ ]:
# Cut down the number of videos from 42000 to 10000 due to low computation power
df = df.head(10000)

In [ ]:
# Exporting dataframes to docs


df = df.head(1)

import docx

# Initialise the Word document
doc = docx.Document()
# Initialise the table
t = doc.add_table(rows=1, cols=df.shape[1])
# Add borders
t.style = 'TableGrid'
# Add the column headings
for j in range(df.shape[1]):
    t.cell(0, j).text = df.columns[j]
# Add the body of the data frame
for i in range(df.shape[0]):
    row = t.add_row()
    for j in range(df.shape[1]):
        cell = df.iat[i, j]
        row.cells[j].text = str(cell)
# Save the Word doc
doc.save('table_1.docx')


We will now create a word soup which is a string that contains all the metadata that we want to feed to our vectorizer viz, overviews, genres, keywords.

In [ ]:
# Run

def create_soup(x):
    return x['genres'] + x['production_companies'] + x['keywords'] +x['actors']

df['word_soup'] = df.apply(create_soup, axis=1)

df.head(2)

,genres,id,original_title,overview,production_companies,keywords,crew,actors,word_soup
0,animation comedy family,862,Toy Story,led woody andy toys live happily room andy bir...,pixar animation studios,jealousy toy boy friendship friends rivalry bo...,"John Lasseter, Joss Whedon, Andrew Stanton, Jo...",tom hanks tim allen rickles jim varney wallace...,animation comedy familypixar animation studios...
1,adventure fantasy family,8844,Jumanji,siblings judy peter discover enchanted board g...,tristar pictures teitler film interscope commu...,board game disappearance based children book n...,"Larry J. Franco, Jonathan Hensleigh, James Hor...",robin williams jonathan hyde kirsten dunst bra...,adventure fantasy familytristar pictures teitl...


In [ ]:
# Run

df['word_soup'].values[0]

'animation comedy familypixar animation studiosjealousy toy boy friendship friends rivalry boy next door new toy toy comes lifetom hanks tim allen rickles jim varney wallace shawn john ratzenberger annie potts john morris erik von detten laurie metcalf r lee ermey sarah freeman penn jillette'

In [ ]:
df.shape

(10000, 9)

# Vectorize the word soup using TfIdf weigted Word2Vec

### Tfidf Vectorizer

In [ ]:
# Run

from sklearn.feature_extraction.text import TfidfVectorizer

model = TfidfVectorizer(min_df=10)# Firstly initialize the Vectorizer and we will consider ony those words which have appeared in atleast 10 overviews.
# min_df=10 is a threshold and we will simply ignore all the words which are below this threshold.

model.fit(df['word_soup']) # Build a unique library based upon movie overviews. The words should have appeared in atleast 10 overviews.
# Suppose d unique words were found, now build a d dimensional vector. In each cell of the vector put the number of occurrences of a particular word in a overeviews.

# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names_out(), list(model.idf_)))
print(dictionary)
#dictionary=sorted(dictionary.items(), key=lambda x:x[1])
#print(dictionary)
print(len(dictionary))
#dictionary=dictionary[:(-5000+1):-1]
#print(dictionary)
#dictionary = dict(dictionary)
#print(dictionary)


# dictionary

{'aaron': 5.290459441148391, 'ab': 7.437851644736734, 'abbott': 6.2401484534243945, 'abdul': 7.437851644736734, 'abe': 6.77645316249137, 'abel': 7.214708093422525, 'abercrombie': 7.1193979136182, 'abigail': 7.725533717188515, 'abo': 7.266001387810076, 'abraham': 6.167389099141966, 'abrahams': 7.437851644736734, 'abrams': 7.645491009514979, 'abuse': 5.961945124927157, 'academy': 7.571383037361257, 'accident': 6.54687872084687, 'accusations': 7.812545094178145, 'accused': 7.165917929253093, 'ackerman': 7.5023901658743055, 'ackland': 6.77645316249137, 'acosta': 7.645491009514979, 'acres': 7.3772270229203, 'action': 2.7750023471619656, 'actor': 6.991564542108315, 'actress': 7.5023901658743055, 'ad': 7.5023901658743055, 'ada': 7.571383037361257, 'adair': 7.437851644736734, 'adam': 4.729801443634525, 'adams': 4.8633328362590476, 'adamson': 7.437851644736734, 'addict': 7.725533717188515, 'addiction': 6.403777877206196, 'addison': 7.3200686090803515, 'addy': 7.214708093422525, 'adele': 7.16591

In [ ]:
# Run

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

import os
from tqdm import tqdm

### Pre-trained Google Word2Vec model

In [ ]:
# Run

# Train your own Word2Vec model using your own text corpus

# Get the list of words in each sentence of word soup

i=0
list_of_words_in_sentence=[] # this is a list of lists
for sentence in df['word_soup']:
    list_of_words_in_sentence.append(sentence.split())

#list_of_words_in_sentence[0]

In [ ]:
is_your_ram_gt_16g=True
want_to_use_google_w2v = True
want_to_train_w2v = False

if want_to_train_w2v: # use this section of code if you want to train your own word2vec model. You can input in the word2vec model medical text if you want to perform nlp medical application.
    # min_count = 5 considers only words that occured atleast 5 times
    w2v_model=Word2Vec(list_of_words_in_sentence,min_count=5,size=50, workers=4)
    print(w2v_model.wv.most_similar('great'))
    print('='*50)
    print(w2v_model.wv.most_similar('worst'))

elif want_to_use_google_w2v and is_your_ram_gt_16g:
    if os.path.isfile('/content/drive/MyDrive/DBMS_project/The_Movie_Dataset/GoogleNews-vectors-negative300.bin'): # in this file we have 300 dim. vector representation for each word in vocabulary
        w2v_model=KeyedVectors.load_word2vec_format('/content/drive/MyDrive/DBMS_project/The_Movie_Dataset/GoogleNews-vectors-negative300.bin', binary=True)
        print(w2v_model.most_similar('great'))
        print(w2v_model.most_similar('worst'))

    else:
        print("you don't have google's word2vec file, keep want_to_train_w2v = True, to train your own w2v ")

[('terrific', 0.7989331483840942), ('fantastic', 0.7935212254524231), ('tremendous', 0.7748855352401733), ('wonderful', 0.7647868990898132), ('good', 0.7291510105133057), ('incredible', 0.7032873630523682), ('marvelous', 0.6971103549003601), ('phenomenal', 0.6841564178466797), ('amazing', 0.663412868976593), ('awesome', 0.6510507464408875)]
[('Worst', 0.6146091818809509), ('weakest', 0.6143776774406433), ('scariest', 0.5957257747650146), ('ugliest', 0.5931181311607361), ('best', 0.5835111141204834), ('bleakest', 0.5718506574630737), ('strongest', 0.567145586013794), ('nastiest', 0.5644308924674988), ('lousiest', 0.563145101070404), ('toughest', 0.5624396204948425)]


In [ ]:
# Save the word2vec model
w2v_model.save('vectors.kv')

In [ ]:
# Run

# Load the word2vec model
w2v_model = KeyedVectors.load('/content/drive/MyDrive/DBMS_project/The_Movie_Dataset/vectors.kv')

In [ ]:
# Run
print(w2v_model.most_similar('great'))

[('terrific', 0.7989331483840942), ('fantastic', 0.7935212254524231), ('tremendous', 0.7748855352401733), ('wonderful', 0.7647868990898132), ('good', 0.7291510105133057), ('incredible', 0.7032873630523682), ('marvelous', 0.6971103549003601), ('phenomenal', 0.6841564178466797), ('amazing', 0.663412868976593), ('awesome', 0.6510507464408875)]


In [ ]:
# Run

# get the 300 dimensional vector
print(w2v_model['computer'])

# Not many values of this 300 dim vector are 0. So this is not a sparse vector but it is a dense vector.

[ 1.07421875e-01 -2.01171875e-01  1.23046875e-01  2.11914062e-01
 -9.13085938e-02  2.16796875e-01 -1.31835938e-01  8.30078125e-02
  2.02148438e-01  4.78515625e-02  3.66210938e-02 -2.45361328e-02
  2.39257812e-02 -1.60156250e-01 -2.61230469e-02  9.71679688e-02
 -6.34765625e-02  1.84570312e-01  1.70898438e-01 -1.63085938e-01
 -1.09375000e-01  1.49414062e-01 -4.65393066e-04  9.61914062e-02
  1.68945312e-01  2.60925293e-03  8.93554688e-02  6.49414062e-02
  3.56445312e-02 -6.93359375e-02 -1.46484375e-01 -1.21093750e-01
 -2.27539062e-01  2.45361328e-02 -1.24511719e-01 -3.18359375e-01
 -2.20703125e-01  1.30859375e-01  3.66210938e-02 -3.63769531e-02
 -1.13281250e-01  1.95312500e-01  9.76562500e-02  1.26953125e-01
  6.59179688e-02  6.93359375e-02  1.02539062e-02  1.75781250e-01
 -1.68945312e-01  1.21307373e-03 -2.98828125e-01 -1.15234375e-01
  5.66406250e-02 -1.77734375e-01 -2.08984375e-01  1.76757812e-01
  2.38037109e-02 -2.57812500e-01 -4.46777344e-02  1.88476562e-01
  5.51757812e-02  5.02929

In [ ]:
# Run

# to find the number of words in the vocabulary for which we have the 300 dim vectors
w2v_words = list(w2v_model.index_to_key)
print(len(w2v_words))

# so in the file we have a big table in which we have 300 dim. vector representation for 30 lakh words

3000000


### Tfidf Weighted Word2Vec model

In [ ]:
tfidf_feat = list(dictionary.keys()) # get the feature names or column names or words of the tfidf vector
print(tfidf_feat)

['aaron', 'ab', 'abbott', 'abdul', 'abe', 'abel', 'abercrombie', 'abigail', 'abo', 'abraham', 'abrahams', 'abrams', 'abuse', 'academy', 'accident', 'accusations', 'accused', 'ackerman', 'ackland', 'acosta', 'acres', 'action', 'actor', 'actress', 'ad', 'ada', 'adair', 'adam', 'adams', 'adamson', 'addict', 'addiction', 'addison', 'addy', 'adele', 'aden', 'adjani', 'adler', 'adlon', 'adolf', 'adolfo', 'adolph', 'adolphe', 'adonis', 'adoption', 'adorf', 'adrian', 'adriana', 'adrien', 'adrienne', 'adult', 'adultery', 'adventure', 'adventurewalt', 'advertising', 'affair', 'affc', 'affleck', 'africa', 'african', 'aftercreditsstinger', 'afterlife', 'ag', 'agar', 'age', 'agent', 'aggression', 'aging', 'agn', 'agnes', 'agust', 'agutter', 'ahmed', 'ahn', 'ai', 'aida', 'aidan', 'aids', 'aiello', 'aileen', 'aim', 'aimee', 'aip', 'air', 'airplane', 'airport', 'aise', 'ajay', 'aki', 'akihiko', 'akiko', 'akim', 'akins', 'akio', 'akira', 'al', 'alain', 'alan', 'alaska', 'alastair', 'alba', 'albert', 'a

In [ ]:
# TF-IDF weighted Word2Vec

tfidf_sent_vectors = []; # the tfidf-weighted w2v for each sentence/review will be stored in this list
row=0;
for sent in tqdm(list_of_words_in_sentence): # # Here sent is a list of words. list_of_words_in_sentence is a list of lists. Each list contains strings of words split from a sentence.
    sent_vec = np.zeros(300) # here we are creating 300 dim vector of zeros. sent_vec will contain the tfidf weighted w2v rep of sentence. It will contain the sum of tfidf(Wi)*W2V(Wi)
    weight_sum =0; # num of words in the sentence which have a valid vector in the google news file. If a word exists in the sentence only then word2vec will generate a vector for that word.
    for word in sent: # sent is a list of words. It represents one sentence.
        if word in w2v_words and word in tfidf_feat: # w2v_words is the list of all the words for which we have 300 dim vectors in the google news file
        # tfidf_feat contains the feature names or column names or words of the tfidf vector
            vec = w2v_model[word] # get 300 dim vector rep of the word. vec is a 300 dim numpy array
            # to reduce the computation we are
            # dictionary[word] = idf value of word in whole corpus
            # sent.count(word) = tf value of word in this review
            tf_idf = (sent.count(word)/len(sent)) * dictionary[word]
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf

    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

100%|██████████| 10000/10000 [2:06:05<00:00,  1.32it/s]


In [ ]:
tfidf_sent_vectors[0]

array([ 0.05101639,  0.07396211,  0.04168974,  0.11525661, -0.01952422,
       -0.042979  , -0.00707238, -0.11475913,  0.03454336,  0.08278292,
       -0.01574087, -0.17462875, -0.0155701 , -0.07967136, -0.14571487,
        0.00196991,  0.00388938,  0.12816896, -0.1449308 , -0.10106924,
        0.06245807, -0.02603116,  0.14078323,  0.06635957, -0.0704737 ,
       -0.00227626, -0.06498832,  0.12510254,  0.17665027, -0.08486828,
       -0.03329963, -0.01766082, -0.07145951,  0.04746875, -0.04142382,
       -0.03314916,  0.10894043,  0.13507259,  0.05291329,  0.06901492,
       -0.03876443, -0.02123908,  0.13351976,  0.05022279,  0.02141149,
        0.02909132,  0.06419093,  0.01949833, -0.00932555,  0.01042309,
       -0.08981006,  0.10210794, -0.00420652, -0.00222924, -0.01542148,
       -0.0485699 ,  0.02799905, -0.07530378, -0.0694432 , -0.08356639,
       -0.06542681,  0.09179146, -0.08371538, -0.0336284 , -0.11099095,
       -0.0698775 , -0.16277302, -0.01973413, -0.05095855,  0.10

### Create a pickel file of the computed tfidf weighted Avd Word2Vec

In [ ]:
file_name = "tfidf_sent_vectors.pkl"

open_file = open(file_name, "wb")
pickle.dump(tfidf_sent_vectors, open_file)
open_file.close()

# Run from here

In [ ]:
file_name = "/content/drive/MyDrive/DBMS_project/The_Movie_Dataset/tfidf_sent_vectors.pkl"
open_file = open(file_name, "rb")
tfidf_avg_w2v = pickle.load(open_file)
open_file.close()

#print(tfidf_avg_w2v )

## Compute Cosine Similarity matrix between movies



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_avg_w2v, tfidf_avg_w2v)

In [ ]:
cosine_sim

array([[1.        , 0.70317954, 0.66270784, ..., 0.65100817, 0.68038079,
        0.26538604],
       [0.70317954, 1.        , 0.85666442, ..., 0.89953686, 0.87950539,
        0.37362589],
       [0.66270784, 0.85666442, 1.        , ..., 0.82591055, 0.7650616 ,
        0.33954029],
       ...,
       [0.65100817, 0.89953686, 0.82591055, ..., 1.        , 0.84729003,
        0.36713437],
       [0.68038079, 0.87950539, 0.7650616 , ..., 0.84729003, 1.        ,
        0.37456044],
       [0.26538604, 0.37362589, 0.33954029, ..., 0.36713437, 0.37456044,
        1.        ]])

In [ ]:
cosine_sim.dtype

dtype('float64')

In [ ]:
cosine_sim[3][160] # This is the similarity between the movie at index 3 and index 160.

0.8038538710016242

In [ ]:
cosine_sim.shape

(10000, 10000)

The more the distance, the less similar the movies are.

## Recommending Similar Movies

In [ ]:
#Construct a pandas series of row indices and movie titles

indices = pd.Series(df.index, index=df['original_title']).drop_duplicates()
indices.head(2)

original_title
Toy Story    0
Jumanji      1
dtype: int64

In [ ]:
indices.shape

(10000,)

In [ ]:

# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    #print(sim_scores)

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # we want most similar movies similarity scores will begin from 1 and will decrease
    #print(sim_scores)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11] # 0th index is the movie itself
    #print(sim_scores)

    # Get the movie indices of top 10 most similar movies
    movie_indices = [i[0] for i in sim_scores]

    df1 = df['original_title'].iloc[movie_indices].to_frame()
    df1.reset_index(drop=True, inplace=True)
    # Convert the list of tuples (sim_scores) into a DataFrame
    df2=pd.DataFrame(sim_scores, columns=['movie_id', 'cosine_similarity'])
    # Merge the new DataFrame with the existing one based on a common index or column
    merged_df = pd.concat([df1, df2['cosine_similarity']], axis=1)
    return merged_df


In [ ]:
get_recommendations('Toy Story', cosine_sim)

,original_title,cosine_similarity
0,Small Soldiers,0.903666
1,Toys,0.897556
2,Child's Play,0.851091
3,Big,0.845291
4,Bride of Chucky,0.811191
5,Pinocchio,0.804937
6,Dolls,0.794354
7,Home Alone 3,0.784195
8,The Santa Clause 2,0.780865
9,Tarzan,0.777688


In [ ]:
get_recommendations('Batman', cosine_sim)

,original_title,cosine_similarity
0,Batman Begins,0.950700
1,The Matrix Reloaded,0.944087
2,Catwoman,0.942959
3,L.A. Confidential,0.941130
4,Superman II,0.939926
5,The Fifth Element,0.938756
6,Daredevil,0.938752
7,Twelve Monkeys,0.937739
8,Westworld,0.937543
9,Mulholland Drive,0.937350
